<h1 style="color:black" align="center">Градиентный бустинг</h1>

Ранее мы изучили бэггинг и случайные леса — подходы к построению композиций, которые независимо обучают каждый базовый алгоритм по некоторому подмножеству обучающих данных. При этом возникает ощущение, что мы используем возможности объединения алгоритмов не в полную силу, и можно было бы строить их так, чтобы каждая следующая модель исправляла ошибки предыдущих. Ниже мы рассмотрим метод, который реализует эту идею — градиентный бустинг, предложенный Фридманом [1]. Он работает для любых дифференцируемых функций потерь и является одним из наиболее мощных и универсальных на сегодняшний день.

<h1 style="color:#008B8B">1. Бустинг в задаче регрессии</h1>

Рассмотрим задачу минимизации квадратичного функционала:

$\large \frac{1}{\ell}
\sum\limits_{i = 1}^{\ell}
    (a(x_i) - y_i)^2
\to
\underset{a}{\text{min}}$

Будем искать итоговый алгоритм в виде суммы *базовых моделей* (weak learners) $b_n(x)$:

$\large a_N(x)
=
\sum\limits_{n = 1}^{N}
    b_n(x)$

где базовые алгоритмы $b_n$ принадлежат некоторому семейству $\mathbb{A}$. Построим первый базовый алгоритм по всем базовым моделям $b \in \mathbb{A}$:

$\large a_1(x) = b_1(x)
:=
\underset{b \in \mathbb{A}}{\text{argmin}}
    \frac{1}{\ell}
    \sum\limits_{i = 1}^{\ell}
        (b(x_i) - y_i)^2$
        
Посчитаем насколько базовая модель ошибается на каждом объете:

$\large s_i^{(1)} = y_i - a_1(x_i)$ - сдвиги


Если прибавить эти остатки к ответам построенного алгоритма, то он не будет допускать ошибок на обучающей выборке. Значит, будет разумно построить второй алгоритм так, чтобы его ответы были как можно ближе к остаткам:

$\large b_2(x)
:=
\underset{b \in \mathbb{A}}{\text{argmin}}
    \frac{1}{\ell}
    \sum\limits_{i = 1}^{\ell}
        (b(x_i) - s_i^{(1)})^2$
        
Если $b_2$ идеальна, тогда мы берем копозицию из двух моделей и она будет всюду выдавать правиьный овет $y_i$:

$\large a_2(x_i) = b_1(x_i) + s_i^{(1)} = b_1(x_i) + y_i - a_1(x_i) = y_i$

Так как зачастую этого недостаточно, тогда мы посчитаем следующие сдвиги:

$\large s_i^{(2)} = y_i - a_2(x_i)$ - сдвиги

Каждый следующий алгоритм тоже будем настраивать на остатки предыдущих:


$\large s_i^{(N)} = 
y_i - \sum\limits_{n = 1}^{N - 1} b_n(x_i)
=
y_i
-
a_{N - 1}(x_i)
\qquad
i = 1, \dots, \ell$

$\large b_N(x)
:=
\underset{b \in \mathbb{A}}{\text{argmin}}
    \frac{1}{\ell}
    \sum\limits_{i = 1}^{\ell}
        (b(x_i) - s_i^{(N)})^2$

<h1 style="color:#008B8B">2. Градиентный бустинг</h1>

Пусть дана некоторая дифференцируемая функция потерь $L(y, z)$. Будем считать, что мы обучили первую модель $b_1(x)$. Как построить следующие модели? Обучение $b_N$ запишем следующую задачу:

$\large \sum\limits_{i = 1}^{\ell} L(y_i, a_{N-1}(x_i) + b_N(x_i)) \to \underset{b_N(x)}{\text{min}}$

Где $a_{N-1}(x_i) = \sum\limits_{n=1}^{N-1} b_n(x)$ композиция из построенной $N-1$ модели.

В этой задаче, мы хотим найти $b_N$ модель такой, что если ее прибавить к построенной композиции, тогда должна получиться более маленькая ошибка. 

Если фукнция потерь удовлетворяет тому, что можно перенести агрумент из одной части в другую $L(y, a+b) = L(y-a, b)$, тогда мы можем дообучать базовую модель на новую целевую переменную:

$\large \sum\limits_{i = 1}^{\ell} L(y_i - a_{N-1}(x_i), b_N(x_i)) \to \underset{b_N(x)}{\text{min}}$, тогда мы получаем задачу обучения базовой модели с обновленной целевой переменной, которая равна разности. 

Но, не все функции потерь способны удовлетворять этому требованию. Поэтому, нам необходим другой подход.

### Другой подход

Ответим в первую очередь на следующий вопрос: если бы в качестве алгоритма $b_N(x)$ мы могли выбрать совершенно любую функцию, то какие значения ей следовало бы принимать на объектах обучающей выборки? Иными словами, нам нужно понять, какие числа $s_1, \ldots, s_{\ell}$ надо выбрать для решения следующей задачи:

$\large \sum\limits_{i = 1}^{\ell}
    L(y_i, a_{N - 1}(x_i) + s_i)
\to
\underset{s_1, \dots, s_\ell}{\text{min}}$


Понятно, что можно требовать $s_i = y_i - a_{N - 1}(x_i)$, но такой подход никак не учитывает особенностей функции потерь $L(y, z)$ и требует лишь точного совпадения предсказаний и истинных ответов. Наприме, имеется не симметричная фукнция потерь, которая по разному штрафует за отклонения, за перепрогноз штраф растет быстрее. Тогда при равной аболютной ошибке штрафы для объектов должны быть разными, следовательно $s_i$ должно на объектах с большим штрафом хотеть больше уменьшить ошибку, так как это внесет большой вклад в ошибку фукнции потреь. Следовательно, мы хотим получить более крупные корректировки для объектво, которые в правой ветке функции потерь, но если использовать текущий вариант, тогда корректировка будет одинаковой для двух объектов, что плохо.

Более разумно потребовать, чтобы сдвиг $s_i$ был противоположен производной функции потерь в точке: $z = a_{N - 1}(x_i)$:

$\large s_i
=
-
\left.
\frac{\partial L}{\partial z}
\right|_{z = a_{N - 1}(x_i)}$

Мы придумали чему должны равняться корректировки, но это на данный момент просто числа, а нам необходимо обучать модели. Так как мы знаем, чему дожна быть равна корректировка в $L$ точках, тогда, обучим базовую модель так, чтобы она была как можно ближе к $s_i$. Если базовая модель будет равняться примено сдвигу, тогда мы сдвиним ошибку в меньшую сторону, так как $b_N$ будет двигать в ту сторону, в которую ошибка будешь уменьшаться. Следовательно, будем обучать очередную базовую модель на следующую задачу:

$\large \frac{1}{\ell}\sum\limits_{i = 1}^{\ell} (b_N(x_i) - s_i)^2 \to \underset{b_N(x)}{\text{min}}$

### Причина обучения на MSE

Почему не обучаем дерево на исходную фукнцию потерь?

$\large \frac{1}{\ell}\sum\limits_{i = 1}^{\ell} L(s_i, b_N(x_i)) \to \underset{b_N(x)}{\text{min}}$

Обучение на MSE может бы . Расскроем квадрат функционала ошибки:

$\large \frac{1}{\ell}\sum\limits_{i = 1}^{\ell} (b_N^2(x_i) - 2 b_N^2(x_i) s_i + s_i^2) \to \underset{b_N(x)}{\text{min}}$

Тогда $s_i^2$ - это конастанта, поэтому можно его выкинуть. $b_N^2(x_i)$ в некоторой степени регуляризатор. И остается $-2 b_N^2(x_i) s_i$, которое мы минимизируем, если убрать минус тогда это необходимо максимизировать. Это выражение является скалярным произведение выходов модели с остатками, то есть, это угол между направлением в котором необходимо сдвигать прогнозы композиции и направлением, которое дерево смогло выучить, и мы хотим чтобы эти направления были как можно сильнее соноправленными.

Получаем, что в бустинге две фукнции потерь, исходная, которая используется для рассчета градинета и ещё одна для обучения базовой модели на градиент.

### Почему это называется градиентным бустингом?

Предположим, что в выборке всего два объекта 

* $\ell = 2$

* $X = \{(x_1, y_1), (x_2, y_2)\}$

Будем на этой выборке обучать бустинг, тогда по оси $x$ будет прогноз на первом объекте $a_N(x_1)$, на оси $y$ будет нарисован прогноз текущей композиции на втором объекте $a_N(x_2)$. И нарисуем линии уровня ошибки этой модели, например, $L(y_1, a_N(x_1)) + L(y_2, a_N(x_2))$.

* Выбираем первую константную модель и ставим точку на графике. 

* Считаем сдвии, то есть, считаем градиент нашего фукнционала $L(y_1, a_N(x_1)) + L(y_2, a_N(x_2))$, по  параметрам $a_N(x_1), a_N(x_2)$. Следовательно, мы считаем куда необходимо менять прогнозы на объектах, чтобы ошибка уменьшилась как можно сильнее. После рассчетов, перемещаемся на графике в сторону сдвига.

* Подбираем модель $b_2$ такой, чтобы она аппроксимировала полученное направление. Дальше мы строим модель и прибавляем к композиции, так как она довльно простая, то сдвинимся немного в другую сторону, переместившись в следующую точку.

* После в этой точке считаем градиент, аппроксимиров его следующей моделью

* ...

Получается, что на каждом шаге мы считаем куда необходимо сдвинуть прогноз композии, аппроксимируем это направление и двигаемся в его направлении. И итоговая композиция моделей $a_N$ приходит в точку оптимума.

Следовательно, градиентный бустинг - это градиентый спуск в пространстве прогнозов композиции на обучающей выборке. 

<h1 style="color:#008B8B">3. Регуляризация</h1>